### Обучение классификатора на определение фронтальных и изображений со спины

In [1]:
# импорт библиотеки
import sys
import os
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../verificator')

In [2]:
from verificator.feature_extractor import InceptionExtractor

In [3]:
from verificator.tpe_predictor import TPEPredictor

Using TensorFlow backend.


In [4]:
inception_weights = 'D:/data/models/inception/classify_image_graph_def.pb'
tpe_weights = 'D:/data/models/final_models/weights/inception/cam_1/cam_1_best.h5'

In [5]:
root_pics_path = 'D:/data/pics/final_sets/clean_cam_1_thr_0.5/train'
marked_folders_numbers = np.linspace(1, 140, 140, dtype = 'int').astype('str') # ПАПКИ, где есть разметка на front/back

In [6]:
data_info = {} # словарь для хранения разметки front/back/neutral
for p_folder in marked_folders_numbers:
    person_folder = os.path.join(root_pics_path, p_folder)
    _files = os.listdir(person_folder)
    frontal_imgs = []
    back_imgs = []
    if 'front' in _files:
        frontal_imgs = os.listdir(os.path.join(person_folder, 'front'))
    if 'back' in _files:
        back_imgs = os.listdir(os.path.join(person_folder, 'back'))
    neutral_imgs = [x for x in _files if os.path.isfile(os.path.join(person_folder,x)) and x not in frontal_imgs and x not in back_imgs]
    data_info[p_folder] = {'front': frontal_imgs, 'back': back_imgs, 'neutral': neutral_imgs}

In [7]:
incepton_extractor = InceptionExtractor(inception_weights)
incepton_extractor.create_graph()

In [8]:
labels = np.array([], dtype = 'int')
classes = np.array([], dtype = 'int')
inception_features = None
for p_folder in marked_folders_numbers:
    person_folder = os.path.join(root_pics_path, p_folder)
    _files = [x for x in os.listdir(person_folder) if os.path.isfile(os.path.join(person_folder, x))]
    files_to_convert = []
    for file in _files:
        labels = np.append(labels, int(p_folder))
        if file in data_info[p_folder]['front']:
            classes = np.append(classes, 1)
        elif file in data_info[p_folder]['back']:
            classes = np.append(classes, 2)
        elif file in data_info[p_folder]['neutral']:
            classes = np.append(classes, 3)
        else:
            raise 'File {} not found'.format(file)
        files_to_convert.append(os.path.join(person_folder, file))
    _features = incepton_extractor.extract_features(files_to_convert)
    if inception_features is None:
        inception_features = _features
    else:
        inception_features = np.vstack([inception_features, _features])
    

In [15]:
inception_features.shape
classes.shape

(4711,)

In [11]:
all_rnd_indices = np.array([], dtype = 'int')
for c_class in np.unique(classes):
    indices = np.where(classes == c_class)[0]
    rnd_indices = np.random.choice(indices.shape[0], int(indices.shape[0]*0.2), replace=False)
    all_rnd_indices = np.append(all_rnd_indices, rnd_indices)
    

In [12]:
all_rnd_indices.shape

(941,)

In [16]:
test_features = inception_features[all_rnd_indices]
test_labels = labels[all_rnd_indices]
test_classes = classes[all_rnd_indices]
train_mask = np.logical_not(np.isin(np.arange(classes.shape[0]), all_rnd_indices))
train_features = inception_features[train_mask]
train_labels = labels[train_mask]
train_classes = classes[train_mask]

In [20]:
# сохраняем данные
saving_path = 'D:/data/models/front_back_classifier/inception_features'
np.save(os.path.join(saving_path, 'test_features.npy'), test_features)
np.save( os.path.join(saving_path, 'test_labels.npy'), test_labels)
np.save( os.path.join(saving_path, 'test_classes.npy'), test_classes)
np.save(os.path.join(saving_path, 'train_features.npy'), train_features)
np.save( os.path.join(saving_path, 'train_labels.npy'),train_labels)
np.save( os.path.join(saving_path, 'train_classes.npy'), test_classes)

D:\data\pics\final_sets\clean_cam_1_thr_0.5\train\141


In [34]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

In [165]:
kfold = KFold(4, shuffle=True)
steps = [('rfc', RandomForestClassifier(max_depth=5, n_estimators=10)),
        ('mlpc', MLPClassifier(alpha=1))]
pipeline = Pipeline(steps=steps)
params = {'rfc__max_depth': [2,3,4,5,6,7,8], 'rfc__n_estimators':[10,20,30]}
grid = GridSearchCV(pipeline, param_grid=params, cv=kfold, scoring=make_scorer(accuracy_score))

In [166]:
grid.fit(train_features, train_classes)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('rfc', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimat...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'rfc__max_depth': [2, 3, 4, 5, 6, 7, 8], 'rfc__n_estimators': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(accuracy_score), verbose=0)

In [169]:
best_estimator_inception_features = grid.best_estimator_

In [170]:
pred_classes = best_estimator_inception_features.predict(test_features)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [94]:
from sklearn.metrics import accuracy_score

In [171]:
accuracy_score(test_classes, prd)

0.74814027630180657

In [227]:
gbc_2 = GradientBoostingClassifier()
gbc_2.fit(train_features, train_123_classes)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [228]:
prd_2 = gbc_2.predict(test_features)

In [229]:
accuracy_score(test_123_classes, prd_2)

0.79064824654622745

In [233]:
prd_2.shape[0]*0.03

28.23

In [172]:
tpe_predictor = TPEPredictor(2048,512)

../verificator\verificator\tpe_model\tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, input_dim=2048, weights=[array([[ ..., activation="linear", use_bias=False)`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
../verificator\verificator\tpe_model\tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
C:\ProgramData\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
../verificator\verificator\tpe_model\tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.T

In [173]:
tpe_predictor.load_weights(tpe_weights)

In [174]:
test_tpe_features = tpe_predictor.predict(test_features)

In [175]:
train_tpe_features = tpe_predictor.predict(train_features)

In [177]:
steps_tpe = [('rfc', RandomForestClassifier(max_depth=5, n_estimators=10))]
pipeline_tpe = Pipeline(steps=steps)
params_tpe = {'rfc__max_depth': [2,3,4,5,6,7,8], 'rfc__n_estimators':[10,20,30]}
grid_tpe = GridSearchCV(pipeline, param_grid=params, cv=kfold, scoring=make_scorer(accuracy_score))

In [178]:
grid_tpe.fit(train_tpe_features, train_classes)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Sup

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('rfc', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimat...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'rfc__max_depth': [2, 3, 4, 5, 6, 7, 8], 'rfc__n_estimators': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(accuracy_score), verbose=0)

In [206]:
grid_tpe.best_params_

{'rfc__max_depth': 7, 'rfc__n_estimators': 30}

In [179]:
best_estimator_tpe_features = grid_tpe.best_estimator_

In [180]:
pred_classes_tpe = best_estimator_tpe_features.predict(test_tpe_features)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [181]:
accuracy_score(test_classes, pred_classes_tpe)

0.57492029755579166

## Пробуем VGG16

In [21]:
from keras.preprocessing import image
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [23]:
vgg16_model = VGG16(weights='imagenet', include_top=False, pooling='max')

C:\ProgramData\Anaconda3\lib\site-packages\keras\applications\vgg16.py:182: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [24]:
labels = np.array([], dtype = 'int')
classes = np.array([], dtype = 'int')
vgg16_features = None
for p_folder in marked_folders_numbers:
    person_folder = os.path.join(root_pics_path, p_folder)
    _files = [x for x in os.listdir(person_folder) if os.path.isfile(os.path.join(person_folder, x))]
    files_to_convert = []
    for file in _files:
        labels = np.append(labels, int(p_folder))
        if file in data_info[p_folder]['front']:
            classes = np.append(classes, 1)
        elif file in data_info[p_folder]['back']:
            classes = np.append(classes, 2)
        elif file in data_info[p_folder]['neutral']:
            classes = np.append(classes, 3)
        else:
            raise 'File {} not found'.format(file)
        files_to_convert.append(os.path.join(person_folder, file))
    img_batch = None
    for img in files_to_convert:
        x = image.load_img(img, target_size=(224, 224))
        x = image.img_to_array(x)
        x = np.expand_dims(x, axis=0)
        if img_batch is None:
            img_batch = x
        else:
            img_batch = np.concatenate((img_batch, x), axis=0)
        
    img_batch = preprocess_input(img_batch)
    _features = vgg16_model.predict(img_batch)
#     print(img_batch.shape)
#     print(_features.shape)
#     break
    if vgg16_features is None:
        vgg16_features = _features
    else:
        vgg16_features = np.vstack([vgg16_features, _features])

In [25]:
test_vgg_features = vgg16_features[all_rnd_indices]
test_vgg_labels = labels[all_rnd_indices]
test_vgg_classes = classes[all_rnd_indices]
train_mask = np.logical_not(np.isin(np.arange(classes.shape[0]), all_rnd_indices))
train_vgg_features = vgg16_features[train_mask]
train_vgg_labels = labels[train_mask]
train_vgg_classes = classes[train_mask]

In [45]:
print(train_vgg_features.shape, train_vgg_classes.shape)
print(test_vgg_features.shape, test_vgg_classes.shape)

(3960, 512) (3960,)
(941, 512) (941,)


In [26]:
# сохраняем данные
saving_path = 'D:/data/models/front_back_classifier/vgg_16_features'
np.save( os.path.join(saving_path, 'test_vgg_features.npy'),test_vgg_features)
np.save( os.path.join(saving_path, 'test_vgg_labels.npy'), test_vgg_labels)
np.save( os.path.join(saving_path, 'test_vgg_classes.npy'), test_vgg_classes)
np.save( os.path.join(saving_path, 'train_vgg_features.npy'), train_vgg_features)
np.save( os.path.join(saving_path, 'train_vgg_labels.npy'), train_vgg_labels)
np.save( os.path.join(saving_path, 'train_vgg_classes.npy'), train_vgg_classes)

In [ ]:
### Присоединяем новые изображения

In [ ]:
new_marked_folders = np.linspace(141, 430, (430-141-1), dtype = 'int').astype('str')

In [96]:
import time

In [102]:
labels = np.array([], dtype='int')
class_labels = np.array([], dtype='int')
features = None
for p_folder in new_marked_folders:
    start = time.time()
    pers_folder = os.path.join(root_pics_path, p_folder)
    img_batch = None
    for label in ['1', '2', '3']:
        cfolder = os.path.join(pers_folder, label)
        cfiles = os.listdir(cfolder)
        for file_ in cfiles:
            labels = np.append(labels, int(p_folder))
            class_labels = np.append(class_labels, int(label))
            x = image.load_img(os.path.join(cfolder, file_), target_size=(224,224))
            x = image.img_to_array(x)
            x = np.expand_dims(x, axis=0)
            if img_batch is None:
                img_batch = x
            else:
                img_batch = np.concatenate((img_batch, x), axis = 0)
    print(p_folder)
    print('--------------\nprocessing {}, time for loading images {}'.format(p_folder, time.time() - start))
    start = time.time()
    img_batch = preprocess_input(img_batch)
    print('processing {}, time for preprocess images {}'.format(p_folder, time.time() - start))
    start = time.time()
    _features = vgg16_model.predict(img_batch)
    finish = time.time()
    print('processing {}, time for getting features {} \n-------------'.format(p_folder, finish - start))
    print('average vgg16 FPS: {}'.format(_features.shape[0]/(finish-start)))
    if features is None:
        features = _features
    else:
        start = time.time()
        features = np.vstack([features, _features])
        print('time for vstacking {}'.format(time.time() - start))

141
--------------
processing 141, time for loading images 0.09024763107299805
processing 141, time for preprocess images 0.009024381637573242
processing 141, time for getting features 1.8319509029388428 
-------------
average vgg16 FPS: 10.371456991298139
142
--------------
processing 142, time for loading images 0.31031250953674316
processing 142, time for preprocess images 0.015837430953979492
processing 142, time for getting features 4.341228008270264 
-------------
average vgg16 FPS: 10.365730598409638
time for vstacking 0.0
143
--------------
processing 143, time for loading images 0.028075218200683594
processing 143, time for preprocess images 0.0030083656311035156
processing 143, time for getting features 1.0737330913543701 
-------------
average vgg16 FPS: 10.244631639437486
time for vstacking 0.0005016326904296875
144
--------------
processing 144, time for loading images 0.30957865715026855
processing 144, time for preprocess images 0.015626192092895508
processing 144, time 

170
--------------
processing 170, time for loading images 0.6531429290771484
processing 170, time for preprocess images 0.015666723251342773
processing 170, time for getting features 4.340130567550659 
-------------
average vgg16 FPS: 10.368351665833783
time for vstacking 0.0010037422180175781
171
--------------
processing 171, time for loading images 0.6942424774169922
processing 171, time for preprocess images 0.0216519832611084
processing 171, time for getting features 4.338660955429077 
-------------
average vgg16 FPS: 10.371863683814784
time for vstacking 0.0
172
--------------
processing 172, time for loading images 0.5854775905609131
processing 172, time for preprocess images 0.015595197677612305
processing 172, time for getting features 4.064082145690918 
-------------
average vgg16 FPS: 10.334436779170897
time for vstacking 0.0010020732879638672
173
--------------
processing 173, time for loading images 0.3134777545928955
processing 173, time for preprocess images 0.015748500

199
--------------
processing 199, time for loading images 0.5477738380432129
processing 199, time for preprocess images 0.0
processing 199, time for getting features 3.6874454021453857 
-------------
average vgg16 FPS: 10.305237327145587
time for vstacking 0.0015053749084472656
200
--------------
processing 200, time for loading images 1.2348408699035645
processing 200, time for preprocess images 0.015662670135498047
processing 200, time for getting features 7.2352540493011475 
-------------
average vgg16 FPS: 10.365911063930678
time for vstacking 0.0
201
--------------
processing 201, time for loading images 0.5533089637756348
processing 201, time for preprocess images 0.015622615814208984
processing 201, time for getting features 3.5828495025634766 
-------------
average vgg16 FPS: 10.326975769852192
time for vstacking 0.0
202
--------------
processing 202, time for loading images 0.5852553844451904
processing 202, time for preprocess images 0.015636444091796875
processing 202, time

228
--------------
processing 228, time for loading images 0.3698282241821289
processing 228, time for preprocess images 0.01566481590270996
processing 228, time for getting features 3.392890691757202 
-------------
average vgg16 FPS: 10.315687471167323
time for vstacking 0.0
229
--------------
processing 229, time for loading images 0.5159015655517578
processing 229, time for preprocess images 0.0
processing 229, time for getting features 3.9759576320648193 
-------------
average vgg16 FPS: 10.31198111100284
time for vstacking 0.003008127212524414
230
--------------
processing 230, time for loading images 0.4644143581390381
processing 230, time for preprocess images 0.0
processing 230, time for getting features 3.6924993991851807 
-------------
average vgg16 FPS: 10.291132344770432
time for vstacking 0.003008604049682617
231
--------------
processing 231, time for loading images 0.5504472255706787
processing 231, time for preprocess images 0.015629053115844727
processing 231, time for

257
--------------
processing 257, time for loading images 0.3686861991882324
processing 257, time for preprocess images 0.015667200088500977
processing 257, time for getting features 2.7942047119140625 
-------------
average vgg16 FPS: 10.378623969943371
time for vstacking 0.003509521484375
258
--------------
processing 258, time for loading images 0.46555256843566895
processing 258, time for preprocess images 0.01566600799560547
processing 258, time for getting features 3.9696831703186035 
-------------
average vgg16 FPS: 10.328280177762744
time for vstacking 0.004010677337646484
259
--------------
processing 259, time for loading images 0.31047773361206055
processing 259, time for preprocess images 0.015671730041503906
processing 259, time for getting features 2.699474334716797 
-------------
average vgg16 FPS: 10.372389779707794
time for vstacking 0.004011869430541992
260
--------------
processing 260, time for loading images 0.5715205669403076
processing 260, time for preprocess i

processing 287, time for getting features 1.0764334201812744 
-------------
average vgg16 FPS: 10.218932071198207
time for vstacking 0.005014657974243164
288
--------------
processing 288, time for loading images 0.6445584297180176
processing 288, time for preprocess images 0.015624761581420898
processing 288, time for getting features 4.816659927368164 
-------------
average vgg16 FPS: 10.380637361566885
time for vstacking 0.004513740539550781
289
--------------
processing 289, time for loading images 0.25702667236328125
processing 289, time for preprocess images 0.0
processing 289, time for getting features 2.228870391845703 
-------------
average vgg16 FPS: 10.319128507492064
time for vstacking 0.004513740539550781
290
--------------
processing 290, time for loading images 0.4294881820678711
processing 290, time for preprocess images 0.015623092651367188
processing 290, time for getting features 3.587221384048462 
-------------
average vgg16 FPS: 10.314389896461474
time for vstackin

316
--------------
processing 316, time for loading images 0.4537539482116699
processing 316, time for preprocess images 0.015627622604370117
processing 316, time for getting features 3.7837302684783936 
-------------
average vgg16 FPS: 10.30728863653477
time for vstacking 0.005012989044189453
317
--------------
processing 317, time for loading images 0.5053796768188477
processing 317, time for preprocess images 0.003008604049682617
processing 317, time for getting features 3.9712026119232178 
-------------
average vgg16 FPS: 10.324328423057736
time for vstacking 0.005514860153198242
318
--------------
processing 318, time for loading images 0.6051795482635498
processing 318, time for preprocess images 0.015671730041503906
processing 318, time for getting features 4.345055103302002 
-------------
average vgg16 FPS: 10.356600533282647
time for vstacking 0.005515098571777344
319
--------------
processing 319, time for loading images 0.2666776180267334
processing 319, time for preprocess 

processing 344, time for getting features 4.34287691116333 
-------------
average vgg16 FPS: 10.361794939278123
time for vstacking 0.0060160160064697266
345
--------------
processing 345, time for loading images 0.8103499412536621
processing 345, time for preprocess images 0.015629053115844727
processing 345, time for getting features 5.585137367248535 
-------------
average vgg16 FPS: 10.384704293956005
time for vstacking 0.0
346
--------------
processing 346, time for loading images 0.5347692966461182
processing 346, time for preprocess images 0.0020105838775634766
processing 346, time for getting features 4.0683417320251465 
-------------
average vgg16 FPS: 10.32361654120269
time for vstacking 0.0060160160064697266
347
--------------
processing 347, time for loading images 0.15801763534545898
processing 347, time for preprocess images 0.0
processing 347, time for getting features 1.2734944820404053 
-------------
average vgg16 FPS: 10.208132177511498
time for vstacking 0.00601744651

373
--------------
processing 373, time for loading images 0.5250263214111328
processing 373, time for preprocess images 0.0
processing 373, time for getting features 4.058914661407471 
-------------
average vgg16 FPS: 10.347593754394449
time for vstacking 0.01566910743713379
374
--------------
processing 374, time for loading images 0.5376162528991699
processing 374, time for preprocess images 0.015623807907104492
processing 374, time for getting features 4.341966390609741 
-------------
average vgg16 FPS: 10.363967832021993
time for vstacking 0.0
375
--------------
processing 375, time for loading images 0.516467809677124
processing 375, time for preprocess images 0.015666484832763672
processing 375, time for getting features 3.876573085784912 
-------------
average vgg16 FPS: 10.318391815357963
time for vstacking 0.007018566131591797
376
--------------
processing 376, time for loading images 0.5713953971862793
processing 376, time for preprocess images 0.015625476837158203
processin

402
--------------
processing 402, time for loading images 0.26578807830810547
processing 402, time for preprocess images 0.019134521484375
processing 402, time for getting features 2.20459246635437 
-------------
average vgg16 FPS: 10.432767212542466
time for vstacking 0.015631675720214844
403
--------------
processing 403, time for loading images 0.10028910636901855
processing 403, time for preprocess images 0.015631914138793945
processing 403, time for getting features 0.41962313652038574 
-------------
average vgg16 FPS: 9.532362855797098
time for vstacking 0.007519721984863281
404
--------------
processing 404, time for loading images 0.5443515777587891
processing 404, time for preprocess images 0.01562952995300293
processing 404, time for getting features 4.353002309799194 
-------------
average vgg16 FPS: 10.337692653803316
time for vstacking 0.008021116256713867
405
--------------
processing 405, time for loading images 0.16333913803100586
processing 405, time for preprocess im

In [107]:
def train_test_split(labels, classes, features, test_part = 0.2):
    assert labels.shape[0] == classes.shape[0] == features.shape[0]
    all_rnd_indices = np.array([], dtype = 'int')
    for c_class in np.unique(classes):
        indices = np.where(classes == c_class)[0]
        rnd_indices = np.random.choice(indices.shape[0], int(indices.shape[0]*0.2), replace=False)
        all_rnd_indices = np.append(all_rnd_indices, rnd_indices)
    test_labels = labels[all_rnd_indices]
    test_classes = classes[all_rnd_indices]
    test_features = features[all_rnd_indices]
    train_mask = np.logical_not(np.isin(np.arange(classes.shape[0]), all_rnd_indices))
    train_labels = labels[train_mask]
    train_classes = classes[train_mask]
    train_features = features[train_mask]
    return train_labels, train_classes, train_features, test_labels, test_classes, test_features

In [108]:
train_labels, train_classes, train_features, test_labels, test_classes, test_features = train_test_split(labels,
                                                                                                         class_labels,
                                                                                                        features)

In [109]:
# сохраняем данные
saving_path = 'D:/data/models/front_back_classifier/vgg_16_features'
np.save( os.path.join(saving_path, 'test_vgg_features_141_430.npy'),test_features)
np.save( os.path.join(saving_path, 'test_vgg_labels_141_430.npy'), test_labels)
np.save( os.path.join(saving_path, 'test_vgg_classes_141_430.npy'), test_classes)
np.save( os.path.join(saving_path, 'train_vgg_features_141_430.npy'), train_features)
np.save( os.path.join(saving_path, 'train_vgg_labels_141_430.npy'), train_labels)
np.save( os.path.join(saving_path, 'train_vgg_classes_141_430.npy'), train_classes)

### Обучаем классификатор на объединенных данных

In [122]:
train_labels = np.load(os.path.join(saving_path, 'train_vgg_labels.npy'))
train_labels = np.append(train_labels, np.load(os.path.join(saving_path, 'train_vgg_labels_141_430.npy')))
train_labels.shape

(12366,)

In [126]:
train_classes = np.load(os.path.join(saving_path, 'train_vgg_classes.npy'))
train_classes = np.append(train_classes, np.load(os.path.join(saving_path, 'train_vgg_classes_141_430.npy')))
train_classes.shape

(12366,)

In [127]:
train_features = np.load(os.path.join(saving_path, 'train_vgg_features.npy'))
train_features = np.vstack([train_features, np.load(os.path.join(saving_path, 'train_vgg_features_141_430.npy'))])
train_features.shape

(12366, 512)

In [129]:
rfc_clf_430 = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', max_depth=5, n_estimators=500)

In [130]:
rfc_clf_430.fit(train_features, train_classes)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [131]:
test_labels = np.load(os.path.join(saving_path, 'test_vgg_labels.npy'))
test_labels = np.append(test_labels, np.load(os.path.join(saving_path, 'test_vgg_labels_141_430.npy')))
test_labels.shape

(2955,)

In [132]:
test_classes = np.load(os.path.join(saving_path, 'test_vgg_classes.npy'))
test_classes = np.append(test_classes, np.load(os.path.join(saving_path, 'test_vgg_classes_141_430.npy')))
test_classes.shape

(2955,)

In [133]:
test_features = np.load(os.path.join(saving_path, 'test_vgg_features.npy'))
test_features = np.vstack([test_features, np.load(os.path.join(saving_path, 'test_vgg_features_141_430.npy'))])
test_features.shape

(2955, 512)

In [134]:
prd = rfc_clf_430.predict(test_features)

In [168]:
prd_p = rfc_clf_430.predict_proba(test_features)

In [178]:
confusions_mask = [pred_ != true_ for pred_, true_ in zip(prd, test_classes)]

In [186]:
confused_features = test_features[confusions_mask]
confused_classes = test_classes[confusions_mask]
confused_features.shape[0]/test_features.shape[0]

0.16209813874788495

In [187]:
pred_c = rfc_clf_430.predict_proba(confused_features)

In [192]:
for i in [1,2,3]:
    print(confused_classes[confused_classes==i].shape)

(90,)
(132,)
(257,)


In [135]:
accuracy_score(test_classes, prd)

0.83790186125211508

In [139]:
mask_1 = np.where(train_classes==1)[0]
mask_2 = np.where(train_classes==2)[0]
mask_3 = np.where(train_classes==3)[0]

In [158]:
mask_1_test = np.where(test_classes==1)[0]
mask_2_test = np.where(test_classes==2)[0]
mask_3_test = np.where(test_classes==3)[0]

In [151]:
train_classes_1_2 = train_classes[np.append(mask_1, mask_2)]
train_features_1_2 = train_features[np.append(mask_1, mask_2)]

In [160]:
test_classes_1_2 = test_classes[np.append(mask_1_test, mask_2_test)]
test_features_1_2 = test_features[np.append(mask_1_test, mask_2_test)]

In [152]:
train_classes_1_3 = train_classes[np.append(mask_1, mask_3)]
train_features_1_3 = train_features[np.append(mask_1, mask_3)]

In [159]:
test_classes_1_3 = test_classes[np.append(mask_1_test, mask_3_test)]
test_features_1_3 = test_features[np.append(mask_1_test, mask_3_test)]

In [153]:
train_classes_2_3 = train_classes[np.append(mask_2, mask_3)]
train_features_2_3 = train_features[np.append(mask_2, mask_3)]

In [161]:
test_classes_2_3 = test_classes[np.append(mask_2_test, mask_3_test)]
test_features_2_3 = test_features[np.append(mask_2_test, mask_3_test)]

In [154]:
gbc_1_2 = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', max_depth=5, n_estimators=500)
gbc_1_3 = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', max_depth=5, n_estimators=500)
gbc_2_3 = GradientBoostingClassifier(learning_rate=0.1, loss='deviance', max_depth=5, n_estimators=500)

In [155]:
gbc_1_2.fit(train_features_1_2, train_classes_1_2)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [162]:
prd_1_2 = gbc_1_2.predict(test_features_1_2)

In [163]:
accuracy_score(test_classes_1_2, prd_1_2)

0.94235436893203883

In [156]:
gbc_1_3.fit(train_features_1_3, train_classes_1_3)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [164]:
prd_1_3 = gbc_1_3.predict(test_features_1_3)

In [165]:
accuracy_score(test_classes_1_3, prd_1_3)

0.88764044943820219

In [157]:
gbc_2_3.fit(train_features_2_3, train_classes_2_3)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [166]:
prd_2_3 = gbc_2_3.predict(test_features_2_3)

In [167]:
accuracy_score(test_classes_2_3, prd_2_3)

0.88570084666039506

### Тест обучения классификатора

In [29]:
gbc_clf = GradientBoostingClassifier()

In [30]:
rfc_clf = RandomForestClassifier(max_depth=7, n_estimators=30)

In [31]:
rfc_clf.fit(train_vgg_features, train_vgg_classes)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [32]:
pred_classes_rfc = rfc_clf.predict(test_vgg_features)

In [35]:
accuracy_score(test_vgg_classes, pred_classes_rfc)

0.69287991498405954

In [37]:
gbc_clf.fit(train_vgg_features, train_vgg_classes)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [38]:
pred_classes_gbc_vgg = gbc_clf.predict(test_vgg_features)

In [39]:
accuracy_score(test_vgg_classes, pred_classes_gbc_vgg)

0.78427205100956432

In [47]:
kfold = KFold(4, shuffle=True)
param_grid_vgg = {'loss': ['deviance'], 'learning_rate': [0.001, 0.01, 0.1],
             'n_estimators': [100, 200], 'max_depth': [3, 5]}
grid_vgg_gbc = GridSearchCV(GradientBoostingClassifier(),
                            param_grid=param_grid_vgg,
                            cv=kfold,
                            scoring=make_scorer(accuracy_score),
                           verbose=2)

In [48]:
grid_vgg_gbc.fit(train_vgg_features, train_vgg_classes)

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100 
[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100, total=  34.7s
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.8s remaining:    0.0s


[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100, total=  34.8s
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100 
[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100, total=  34.7s
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100 
[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=100, total=  34.7s
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200 
[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200, total= 1.2min
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200 
[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200, total= 1.2min
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200 
[CV]  learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200, total= 1.2min
[CV] learning_rate=0.001, loss=deviance, max_depth=3, n_estimators=200 
[CV]  learning_rate=0.001, loss=deviance, max_

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 57.6min finished


GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'loss': ['deviance'], 'learning_rate': [0.001, 0.01, 0.1], 'n_estimators': [100, 200], 'max_depth': [3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(accuracy_score), verbose=2)

In [49]:
grid_vgg_gbc.best_params_

{'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 5, 'n_estimators': 200}

In [50]:
grid_vgg_gbc.best_score_

0.83888888888888891

In [52]:
param_grid_vgg_2 = {'loss': ['deviance'], 'learning_rate': [0.01, 0.1],
             'n_estimators': [200, 300, 500, 700, 1000], 'max_depth': [ 5, 7, 9]}
grid_vgg_gbc_2 = GridSearchCV(GradientBoostingClassifier(),
                            param_grid=param_grid_vgg_2,
                            cv=kfold,
                            scoring=make_scorer(accuracy_score),
                           verbose=2)

In [53]:
grid_vgg_gbc_2.fit(train_vgg_features, train_vgg_classes)

Fitting 4 folds for each of 30 candidates, totalling 120 fits
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200, total= 2.0min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.0min remaining:    0.0s


[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200, total= 2.0min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200, total= 2.7min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=200, total= 2.0min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300, total= 3.0min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300, total= 3.0min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300, total= 3.0min
[CV] learning_rate=0.01, loss=deviance, max_depth=5, n_estimators=300 
[CV]  learning_rate=0.01, loss=deviance, max_depth=5, n_es

[CV]  learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=700, total=17.7min
[CV] learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=700 
[CV]  learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=700, total=17.7min
[CV] learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000 
[CV]  learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000, total=20.8min
[CV] learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000 
[CV]  learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000, total=20.7min
[CV] learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000 
[CV]  learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000, total=20.8min
[CV] learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000 
[CV]  learning_rate=0.01, loss=deviance, max_depth=9, n_estimators=1000, total=20.5min
[CV] learning_rate=0.1, loss=deviance, max_depth=5, n_estimators=200 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=

[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=300, total= 2.1min
[CV] learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500, total= 2.1min
[CV] learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500, total= 2.1min
[CV] learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500, total= 2.1min
[CV] learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=500, total= 2.1min
[CV] learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=700 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=700, total= 2.1min
[CV] learning_rate=0.1, loss=deviance, max_depth=9, n_estimators=700 .
[CV]  learning_rate=0.1, loss=deviance, max_depth=9, n_estimator

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 698.4min finished


GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'loss': ['deviance'], 'learning_rate': [0.01, 0.1], 'n_estimators': [200, 300, 500, 700, 1000], 'max_depth': [5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(accuracy_score), verbose=2)

In [55]:
grid_vgg_gbc_2.best_params_

{'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 5, 'n_estimators': 500}

In [57]:
# root_pics_path = 'D:/data/pics/final_sets/clean_cam_1_thr_0.5/train'
unmarked_folders_numbers = np.linspace(141, len(os.listdir(root_pics_path)),
                                     len(os.listdir(root_pics_path))-140, dtype = 'int').astype('str') # ПАПКИ, где есть разметка на front/back

In [59]:
clf_vgg_gbc_2 = grid_vgg_gbc_2.best_estimator_

In [73]:
root_pics_path = 'D:\\data\\pics\\final_sets\\clean_cam_1_thr_0.5\\train'

In [67]:
markup_ary = {}
with open(os.path.join('D:/data/models/front_back_classifier/vgg_16_features', 'markup.txt'), 'w') as f:
    for p_folder in unmarked_folders_numbers:
        files = os.listdir(os.path.join(root_pics_path, p_folder))
        img_batch = None
        for file in files:
            img_file = os.path.join(root_pics_path, p_folder, file)
            x = image.load_img(img_file, target_size=(224, 224))
            x = image.img_to_array(x)
            x = np.expand_dims(x, axis=0)
            if img_batch is None:
                img_batch = x
            else:
                img_batch = np.concatenate((img_batch, x), axis=0)
        img_batch = preprocess_input(img_batch)
        _features = vgg16_model.predict(img_batch)
        _classes = clf_vgg_gbc_2.predict(_features)
        for filename, klass in zip(files, _classes):
            markup_ary[os.path.join(p_folder, filename)] = klass
            f.write('{} {} \n'.format(os.path.join(p_folder, filename), str(klass)))
        print('{} processed'.format(p_folder))
        
            

141 processed
142 processed
143 processed
144 processed
145 processed
146 processed
147 processed
148 processed
149 processed
150 processed
151 processed
152 processed
153 processed
154 processed
155 processed
156 processed
157 processed
158 processed
159 processed
160 processed
161 processed
162 processed
163 processed
164 processed
165 processed
166 processed
167 processed
168 processed
169 processed
170 processed
171 processed
172 processed
173 processed
174 processed
175 processed
176 processed
177 processed
178 processed
179 processed
180 processed
181 processed
182 processed
183 processed
184 processed
185 processed
186 processed
187 processed
188 processed
189 processed
190 processed
191 processed
192 processed
193 processed
194 processed
195 processed
196 processed
197 processed
198 processed
199 processed
200 processed
201 processed
202 processed
203 processed
204 processed
205 processed
206 processed
207 processed
208 processed
209 processed
210 processed
211 processed
212 pr

In [76]:
from shutil import copyfile, copy

In [77]:
for p_folder in unmarked_folders_numbers:
    c_folder = os.path.join(root_pics_path, p_folder)
    c_dict = {k: v for k,v in markup_ary.items() if k.split('\\')[0] == p_folder}
    for c in ['1', '2', '3']:
        if not os.path.exists(os.path.join(c_folder, c)):
            os.mkdir(os.path.join(c_folder, c))
    for c in ['1', '2', '3']:
        for k,v in c_dict.items():
            if str(v) == c:
                copy(os.path.join(root_pics_path, k), os.path.join(c_folder, c))
        

## Закончил на 430

####  Пробуем деление на back и не - back

In [122]:
two_classes = classes

In [125]:
two_classes[np.argwhere(two_classes == 3)] = 1
    

In [127]:
np.unique(two_classes)

array([1, 2])